In [1]:
!pip install translate-toolkit transformers sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.3 MB/s 
     |████████████████████████████████| 4.4 MB 52.7 MB/s 
     |████████████████████████████████| 1.2 MB 58.2 MB/s 
     |████████████████████████████████| 6.4 MB 35.2 MB/s 
     |████████████████████████████████| 101 kB 11.1 MB/s 
     |████████████████████████████████| 6.6 MB 39.3 MB/s 
     |████████████████████████████████| 596 kB 66.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
!wget -O dataset.txt.zip https://opus.nlpl.eu/download.php?f=TEP/v1/moses/en-fa.txt.zip
!unzip dataset.txt.zip


--2022-07-14 00:08:33--  https://opus.nlpl.eu/download.php?f=TEP/v1/moses/en-fa.txt.zip
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-TEP/v1/moses/en-fa.txt.zip [following]
--2022-07-14 00:08:35--  https://object.pouta.csc.fi/OPUS-TEP/v1/moses/en-fa.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16353318 (16M) [application/zip]
Saving to: ‘dataset.txt.zip’

dataset.txt.zip     100%[===================>]  15.60M  10.3MB/s    in 1.5s    

2022-07-14 00:08:37 (10.3 MB/s) - ‘dataset.txt.zip’ saved [16353318/16353318]

Archive:  dataset.txt.zip
  inflating: TEP.en-fa.en            
  inflating: TEP.en-fa.fa            
  inflating: READ

In [3]:
from google.colab import drive
from IPython.display import display
from IPython.html import widgets
import matplotlib.pyplot as plt
import numpy as np
import random
import seaborn as sns
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook

sns.set()

/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:

with open("TEP.en-fa.en", 'rb') as enfile:
  en_lines_list = [line.decode("UTF-8").rstrip() for line in enfile]

with open('TEP.en-fa.fa') as fafile:
  fa_lines_list = [line.rstrip() for line in fafile]

# { sentence_id: 1, translation: {"en": "hello", "fa": "سلام"}
sentences = []
for i in range(0, len(en_lines_list)):
  sentences.append({
        "en": en_lines_list[i],
        "fa": fa_lines_list[i]
  })

print("len en lines: ", len(en_lines_list))
print("len fa lines: ", len(fa_lines_list))
print("first en sentence: ", en_lines_list[0])
print("first fa sentence: ", fa_lines_list[0])

len en lines:  612086
len fa lines:  612086
first en sentence:  raspy breathing .
first fa sentence:  صداي خر خر .


In [5]:
sentences[2:6]

[{'en': 'maybe its the wind .', 'fa': 'شايد صداي باد باشه .'},
 {'en': 'no .', 'fa': 'نه .'},
 {'en': 'stop please stop .',
  'fa': 'دست نگه داريد خواهش ميکنم دست نگه داريد .'},
 {'en': 'you have a week , evans then well burn the house .',
  'fa': 'اوانز تو فقط يک هفته وقت داري وگرنه خونتو خواهيم سوزوند .'}]

In [6]:
random.shuffle(sentences)
train_slice = int(len(sentences) * 0.8)
train_dataset = sentences[:train_slice]
test_dataset = sentences[train_slice:]

In [7]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
model_repo = 'google/mt5-small'
model_path = "/content/gdrive/MyDrive/mt5_translation.pt"
max_seq_len = 20
# model.config.maxlength = 40